In [1]:
import pandas as pd
import os

In [2]:
s3_prefix = "s3://aegovan-data/pubmed_asbtract/predictions_multi_ppi-bert-2021-01-02-08_m_2021010514/"
s3_output_prefix = "{}_summary/".format(s3_prefix.rstrip("/"))
s3_data ="s3://aegovan-data/human_output/human_interactions_ppi_v2.json"

In [3]:
local_temp = "temp"
local_temp_pred_dir = os.path.join( local_temp, "pred_results")
local_temp_wk_dir = os.path.join( local_temp, "wk")

In [4]:
!rm -rf $local_temp
!mkdir -p $local_temp_pred_dir
!mkdir -p $local_temp_wk_dir

In [5]:
import boto3
import glob
from multiprocessing.dummy import Pool as ThreadPool
import argparse
import datetime 
import os


def upload_file(localpath, s3path):
        """
Uploads a file to s3
        :param localpath: The local path
        :param s3path: The s3 path in format s3://mybucket/mydir/mysample.txt
        """

        bucket, key = get_bucketname_key(s3path)

        if key.endswith("/"):
            key = "{}{}".format(key, os.path.basename(localpath))
        
        s3 = boto3.client('s3')
        
        s3.upload_file(localpath, bucket, key)

def get_bucketname_key(uripath):
    assert uripath.startswith("s3://")

    path_without_scheme = uripath[5:]
    bucket_end_index = path_without_scheme.find("/")

    bucket_name = path_without_scheme
    key = "/"
    if bucket_end_index > -1:
        bucket_name = path_without_scheme[0:bucket_end_index]
        key = path_without_scheme[bucket_end_index + 1:]

    return bucket_name, key


def download_file(s3path, local_dir):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')
    
    local_file = os.path.join(local_dir, s3path.split("/")[-1])
    

    s3.download_file(bucket, key, local_file)
    
def download_object(s3path):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')    

    s3_response_object = s3.get_object(Bucket=bucket, Key=key)
    object_content = s3_response_object['Body'].read()
    
    return len(object_content)



def list_files(s3path_prefix):
    assert s3path_prefix.startswith("s3://")
    assert s3path_prefix.endswith("/")
    
    bucket, key = get_bucketname_key(s3path_prefix)
    
   
   
    s3 = boto3.resource('s3')
    
    bucket = s3.Bucket(name=bucket)

    return ( (o.bucket_name, o.key) for o in bucket.objects.filter(Prefix=key))





def upload_files(local_dir, s3_prefix, num_threads=20):    
    input_tuples = ( (f,  s3_prefix) for f in glob.glob("{}/*".format(local_dir)))
    
    with ThreadPool(num_threads) as pool:
        pool.starmap(uploadfile, input_tuples)
    


def download_files(s3_prefix, local_dir, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        
        

def download_objects(s3_prefix, num_threads=20):    
    s3_files = ( "s3://{}/{}".format(s3_bucket,s3_key) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.map(download_object, s3_files)
        
    return sum(results)/1024
        

def get_directory_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def get_s3file_size(bucket, key):
    s3 = boto3.client('s3')
    response = s3.head_object(Bucket=bucket, Key=key)
    size = response['ContentLength']
    return size
    
def download_files_min_files(s3_prefix, local_dir, min_file_size=310, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix) if get_s3file_size(s3_bucket, s3_key) > min_file_size )
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        


In [6]:
%%time

download_files(s3_prefix, local_temp_pred_dir)

CPU times: user 5min 51s, sys: 4min 37s, total: 10min 29s
Wall time: 6min 12s


In [7]:
!ls -l $local_temp_dir | wc -l

20


In [8]:
threshold_config_array = [{'InteractionType': 'phosphorylation', 'Threshold': 0.73911322481},
 {'InteractionType': 'dephosphorylation', 'Threshold': 0.96703105569},
 {'InteractionType': 'acetylation', 'Threshold': 0.8592761814600001},
 {'InteractionType': 'demethylation', 'Threshold': 0.52774533063},
 {'InteractionType': 'methylation', 'Threshold': 0.7215910189300001},
 {'InteractionType': 'ubiquitination', 'Threshold': 0.5717439842500001},
 {'InteractionType': 'deubiquitination', 'Threshold': 0.36090918539000005}]

In [9]:
threshold_config = { r["InteractionType"]:r["Threshold"] for r in threshold_config_array}

In [10]:
%%time 

full_df = None
total_counts = {}
for f in os.listdir(local_temp_pred_dir):
    df = pd.read_json(os.path.join(local_temp_pred_dir, f), orient="records", lines=True )
    
    count_dict = df.groupby(["predicted"])["predicted"].count().to_dict()
    min_dict = df.groupby(["predicted"])["predicted_confidence"].min().to_dict()
    
    for k in count_dict:
        if k not in total_counts:
            total_counts[k] = {}
        total_counts[k]["count"] = total_counts[k].get("count", 0) + count_dict[k]
        total_counts[k]["min"] = min(total_counts[k].get("min", 1.0) , min_dict[k])



    # Filter below threshold items
    high_quality_frames = []
    for k,t in threshold_config.items():
        high_quality_frames.append(df.query("predicted == '{}' and predicted_confidence > {}".format(k, t)))
        
    high_quality_df = pd.concat(high_quality_frames)
    
    
    
    if full_df is None:
        full_df = high_quality_df
    else:
        full_df = pd.concat([high_quality_df, full_df])
        
    
    


CPU times: user 11min 27s, sys: 1min 59s, total: 13min 27s
Wall time: 15min 12s


In [11]:
total_counts

{'acetylation': {'count': 885, 'min': 0.18810680970000002},
 'dephosphorylation': {'count': 4169, 'min': 0.1964187006},
 'methylation': {'count': 3151, 'min': 0.1840680047},
 'other': {'count': 9066267, 'min': 0.18516493380000001},
 'phosphorylation': {'count': 88356, 'min': 0.19867934110000002},
 'ubiquitination': {'count': 4762, 'min': 0.18264465870000002},
 'deubiquitination': {'count': 128, 'min': 0.1857735973}}

In [12]:
df_counts = pd.DataFrame( [ {"InteractionType":k, "Count": v["count"], "MinThreshold":v["min"]}  for k,v in total_counts.items()]) 

In [13]:
df_counts

,InteractionType,Count,MinThreshold
0,acetylation,885,0.188107
1,dephosphorylation,4169,0.196419
2,methylation,3151,0.184068
3,other,9066267,0.185165
4,phosphorylation,88356,0.198679
5,ubiquitination,4762,0.182645
6,deubiquitination,128,0.185774


In [14]:
full_df.groupby(["predicted"])["predicted"].count().to_dict()

{'acetylation': 22,
 'dephosphorylation': 16,
 'deubiquitination': 28,
 'methylation': 300,
 'phosphorylation': 11919,
 'ubiquitination': 104}

In [15]:
full_df.groupby(["predicted"])["predicted_confidence"].describe()

,count,mean,std,min,25%,50%,75%,max
predicted,,,,,,,,
acetylation,22.0,0.902486,0.026082,0.866076,0.881072,0.904164,0.919227,0.951532
dephosphorylation,16.0,0.973829,0.003665,0.967915,0.971103,0.972859,0.977132,0.979208
deubiquitination,28.0,0.455799,0.092871,0.367052,0.391047,0.425471,0.478622,0.763984
methylation,300.0,0.794552,0.055733,0.722113,0.745691,0.782870,0.827493,0.949326
phosphorylation,11919.0,0.834559,0.074557,0.739130,0.772317,0.814921,0.885646,0.995965
ubiquitination,104.0,0.613652,0.038614,0.571816,0.581698,0.599881,0.632836,0.745508


In [16]:
full_df.shape

(12389, 18)

In [17]:
download_file(s3_data, local_temp_wk_dir)

In [18]:
data_file = os.path.join(local_temp_wk_dir, s3_data.split("/")[-1])
data_training_full_df = pd.read_json(data_file)

In [19]:
data_training_full_df.shape

(3381, 7)

In [20]:
full_df.shape

(12389, 18)

In [21]:
data_training_full_df.head(n=2)

,interactionId,interactionType,isNegative,participants,pubmedId,pubmedTitle,pubmedabstract
0,1585513,phosphorylation,false,"[{'uniprotid': 'Q10728', 'alias': [['mypt1_rat...",17126281,None,Zipper-interacting protein kinase (ZIP kinase)...
1,1585516,phosphorylation,false,"[{'uniprotid': 'O43293-1', 'alias': [['o43293-...",17126281,None,Zipper-interacting protein kinase (ZIP kinase)...


In [22]:
data_training_full_df["participants"].sample(n=2).iloc[0]

[{'uniprotid': 'Q13224',
  'alias': [['nmde2_human'],
   ['Glutamate receptor ionotropic, NMDA 2B'],
   ['GRIN2B'],
   ['NMDAR2B'],
   ['N-methyl D-aspartate receptor subtype 2B'],
   ['N-methyl-D-aspartate receptor subunit 3'],
   ['Glutamate [NMDA] receptor subunit epsilon-2']],
  'alternative_uniprots': ['Q13225', 'Q14CU4', 'Q12919', 'Q13220', 'Q9UM56']},
 {'uniprotid': 'P08575',
  'alias': [['ptprc_human'],
   ['Receptor-type tyrosine-protein phosphatase C'],
   ['PTPRC'],
   ['CD45'],
   ['T200'],
   ['Leukocyte common antigen']],
  'alternative_uniprots': ['A0A0A0MT22',
   'Q16614',
   'X6R433',
   'A8K7W6',
   'Q9H0Y6']}]

In [23]:
def get_partipants_key_raw(participants):
    """
    Example input
    [{'uniprotid': 'P19388',
  'alias': [['rpab1_human'],
   ['DNA-directed RNA polymerases I, II, and III subunit RPABC1'],
   ['POLR2E'],
   ['DNA-directed RNA polymerase II subunit E'],
   ['RPB5 homolog'],
   ['DNA-directed RNA polymerase II 23 kDa polypeptide'],
   ['XAP4']],
  'alternative_uniprots': ['Q6PIH5', 'Q9BT06', 'O43380', 'B2R6L4', 'D6W5Y1']},
 {'uniprotid': 'Q96SB4',
  'alias': [['srpk1_human'],
   ['SRSF protein kinase 1'],
   ['Serine/arginine-rich protein-specific kinase 1'],
   ['SFRS protein kinase 1'],
   ['SRPK1']],
  'alternative_uniprots': ['Q5R365', 'Q5R364', 'B4DS61', 'Q8IY12', 'Q12890']}]
    """
    participant_uniprot = []
    for p in participants:
        
        participant_uniprot.append(str(p["uniprotid"]))
        
        
    result = get_partipants_key(participant_uniprot)
    
    return result

def get_partipants_key(list_uniprot):
    participant_uniprot=sorted(filter (lambda x: x is not None, list_uniprot))
    
    result = "#".join(participant_uniprot)
    
    return result


In [24]:
training_participants = data_training_full_df["participants"].apply(get_partipants_key_raw)

In [25]:
full_df_particpiants = full_df[["participant2Id", "participant1Id"]].apply(lambda x : get_partipants_key([x["participant2Id"],x["participant2Id"]]), axis=1)

In [26]:
full_df.head(n=2)

,abstract,annotations,gene_id_map,normalised_abstract,participant1Id,participant2Id,pubmedId,predicted,confidence_scores,acetylation,demethylation,dephosphorylation,deubiquitination,methylation,other,phosphorylation,ubiquitination,predicted_confidence
869,Congenital anomalies of the kidneys and urinar...,"[{'start': '380', 'end': '387', 'name': 'patie...","{'23216': 'Q86TI0', '6517': 'P14672'}",Congenital anomalies of the kidneys and urinar...,P14672,Q86TI0,26572137,phosphorylation,"{'acetylation': 0.0013321067, 'demethylation':...",0.001332,0.000697,0.103006,0.001065,0.001237,0.019506,0.870871,0.002286,0.870871
1770,Spinal muscular atrophy (SMA) is a devastating...,"[{'start': '101', 'end': '105', 'name': 'SMN1'...","{'6606': 'Q16637', '6607': 'Q16637', '5358': '...",Spinal muscular atrophy (SMA) is a devastating...,Q16637,Q16637,26573968,phosphorylation,"{'acetylation': 0.0009106977, 'demethylation':...",0.000911,0.000538,0.009467,0.001097,0.000999,0.189021,0.794962,0.003006,0.794962


In [27]:
full_df.shape

(12389, 18)

In [28]:
full_df[~full_df_particpiants.isin(training_participants)].shape

(12176, 18)

In [29]:
full_df["PubmedInTrainingData"] = full_df_particpiants.isin(training_participants)

In [44]:
tmp_df = pd.DataFrame(full_df[~full_df.PubmedInTrainingData]\
      .groupby('predicted')["predicted"].count())\
      .merge(df_counts.query("InteractionType != 'other'")[["Count", "InteractionType"]], left_index=True, right_on = ["InteractionType"], how="right")\
      [["InteractionType", "Count", "predicted"]]

tmp_df.loc["Sum"] =    ["Total", tmp_df.Count.sum(), tmp_df.predicted.sum() ]
print(tmp_df.to_latex(columns=  ["InteractionType", "Count", "predicted"], index=False)
     )

\begin{tabular}{lrr}
\toprule
   InteractionType &   Count &  predicted \\
\midrule
       acetylation &     885 &         22 \\
 dephosphorylation &    4169 &         10 \\
       methylation &    3151 &        299 \\
   phosphorylation &   88356 &      11714 \\
    ubiquitination &    4762 &        104 \\
  deubiquitination &     128 &         27 \\
             Total &  101451 &      12176 \\
\bottomrule
\end{tabular}



In [31]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

In [32]:
full_df.sample(n=2)

,abstract,annotations,gene_id_map,normalised_abstract,participant1Id,participant2Id,pubmedId,predicted,confidence_scores,acetylation,demethylation,dephosphorylation,deubiquitination,methylation,other,phosphorylation,ubiquitination,predicted_confidence,PubmedInTrainingData
3336,Transforming growth factor-beta (TGF-beta ) ha...,"[{'start': '0', 'end': '31', 'name': 'Transfor...","{'7040': 'P01137', '6497': 'P12755', '4087': '...",P01137 (P01137 ) has dual and paradoxical func...,P01137,P12755,12793438,phosphorylation,"{'acetylation': 0.0006816332, 'demethylation':...",0.000682,0.000387,0.002126,0.000728,0.000510,0.098911,0.895351,0.001305,0.895351,False
13287,MEF2B encodes a transcriptional activator and ...,"[{'start': '0', 'end': '5', 'name': 'MEF2B', '...","{'100271849': 'Q02080', '604': 'P41182', '2352...",Q02080 encodes a transcriptional activator and...,P41182,Q02080,23974956,phosphorylation,"{'acetylation': 0.0013243229, 'demethylation':...",0.001324,0.001256,0.001815,0.003152,0.001275,0.031059,0.908271,0.051847,0.908271,False


In [33]:
full_df["unique_gene_count"] = full_df["gene_id_map"].apply(lambda x: len(x))

In [34]:
full_df.groupby("predicted")[["predicted_confidence","unique_gene_count"]].describe()

predicted_confidence                                \
                                 count      mean       std       min   
predicted                                                              
acetylation                       22.0  0.902486  0.026082  0.866076   
dephosphorylation                 16.0  0.973829  0.003665  0.967915   
deubiquitination                  28.0  0.455799  0.092871  0.367052   
methylation                      300.0  0.794552  0.055733  0.722113   
phosphorylation                11919.0  0.834559  0.074557  0.739130   
ubiquitination                   104.0  0.613652  0.038614  0.571816   

                                                          unique_gene_count  \
                        25%       50%       75%       max             count   
predicted                                                                     
acetylation        0.881072  0.904164  0.919227  0.951532              22.0   
dephosphorylation  0.971103  0.972859  0.977132  0.979208              16.0   
deubiquitination   0.391047  0.425471  0.478622  0.763984              28.0   
methylation        0.745691  0.782870  0.827493  0.949326             300.0   
phosphorylation    0.772317  0.814921  0.885646  0.995965           11919.0   
ubiquitination     0.581698  0.599881  0.632836  0.745508             104.0   

                                                                   
                       mean       std  min   25%  50%   75%   max  
predicted                                                          
acetylation        3.272727  1.162174  2.0  2.25  3.0  4.00   6.0  
dephosphorylation  5.625000  2.305790  2.0  3.75  6.0  8.00   8.0  
deubiquitination   3.000000  1.154701  2.0  2.00  3.0  3.25   6.0  
methylation        3.130000  1.543061  2.0  2.00  3.0  4.00  14.0  
phosphorylation    4.827922  5.166632  2.0  2.00  4.0  6.00  76.0  
ubiquitination     2.884615  1.054879  2.0  2.00  3.0  3.00   7.0

In [35]:
predictions_above_threshold_file = "predictions_above_threshold.json"
full_df.to_json(predictions_above_threshold_file, orient='records')

In [36]:
upload_file(predictions_above_threshold_file, "{}/".format(s3_output_prefix.rstrip("/")))

In [37]:
samples_subset = full_df.query("PubmedInTrainingData == False")\
                .groupby('predicted', group_keys=False)\
                .apply(lambda x: x.sample(min(len(x), 10),random_state=45))

samples_subset.groupby(["predicted"])["predicted"].count()

predicted
acetylation          10
dephosphorylation    10
deubiquitination     10
methylation          10
phosphorylation      10
ubiquitination       10
Name: predicted, dtype: int64

In [38]:
samples_subset.columns

Index(['abstract', 'annotations', 'gene_id_map', 'normalised_abstract',
       'participant1Id', 'participant2Id', 'pubmedId', 'predicted',
       'confidence_scores', 'acetylation', 'demethylation',
       'dephosphorylation', 'deubiquitination', 'methylation', 'other',
       'phosphorylation', 'ubiquitination', 'predicted_confidence',
       'PubmedInTrainingData', 'unique_gene_count'],
      dtype='object')

In [39]:
import json
import json
def create_manifest_file(df, outfile):
    items = df.to_dict(orient='records' )
    with open(outfile , "w") as f:
        for item in items:
            # Write without new lines
            item_m  = {}
            item_m["source"] = json.dumps(item)
            f.write(json.dumps(item_m).replace("\n", "\t"))
            f.write("\n")

In [40]:
samples_subset_file = "predictions_sample_subset.json"
samples_subset.to_json(samples_subset_file, orient='records')
upload_file(samples_subset_file, "{}/".format(s3_output_prefix.rstrip("/")))


manifest_file = "predictions_sample_subset.mainfest"
create_manifest_file(samples_subset, manifest_file)
upload_file(manifest_file, "{}/".format(s3_output_prefix.rstrip("/")))

# Create one manifest file per interaction type
for i in list(samples_subset["predicted"].unique()):
    manifest_file = "predictions_sample_subset_{}.mainfest".format(i)
    create_manifest_file( samples_subset.query("predicted == '{}'".format(i)), manifest_file)
    upload_file(manifest_file, "{}/".format(s3_output_prefix.rstrip("/")))

